In [5]:
import cv2
import mediapipe as mp
import util
import pyautogui
import random
import os
from pynput.mouse import Button, Controller

mouse = Controller()
screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands=mpHands.Hands(static_image_mode=False,
                    model_complexity=1,
                    min_detection_confidence=0.7,
                    min_tracking_confidence=0.7,
                    max_num_hands=1
               )

def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks= processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]

    return none 
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x= int(index_finger_tip.x * screen_width)
        y= int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x,y)
    
def is_left_click(landmarks_list,thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and  # bend index finger
           util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])< 90  and   # straight middle finger
           thumb_index_dist > 50
          )

def is_right_click(landmarks_list,thumb_index_dist):
    return(util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12]) < 50 and  # bend middle finger
           util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) > 90  and
           thumb_index_dist > 50
          )

def is_double_click(landmarks_list,thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
           util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50  and
           thumb_index_dist > 50
          )
    
    
# def is_screenshot(landmarks_list,thumb_index_dist):
#     return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
#            util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50  and
#            thumb_index_dist < 50
#           )

def is_screenshot(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[17], landmarks_list[18], landmarks_list[20]) < 50 and  # Pinky finger bent
            thumb_index_dist < 50)    # Thumb close to index finger


# def is_sleep(landmarks_list,thumb_index_dist):
#     return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
#            util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50  and
#            thumb_index_dist < 50
#           )

def is_sleep(landmarks_list, thumb_index_dist):
    return (thumb_index_dist < 50)  # Thumb close to index finger


def is_shutdown(landmarks_list,thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
           util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50  and
           thumb_index_dist < 50
          )


def detect_gestures(frame,landmarks_list,processed):
    if len(landmarks_list)>=21:
        index_finger_tip =find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmarks_list[4],landmarks_list[5]])
        
        if thumb_index_dist < 50 and util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])>90:
            move_mouse(index_finger_tip)
         # left click
        elif is_left_click(landmarks_list,thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame,'Left Click',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            
         # right click
        elif is_right_click(landmarks_list,thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame,'Right Click',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
         
         # double click
        elif is_double_click(landmarks_list,thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame,'Double Click',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2)
         # Screenshot click 
        elif is_screenshot(landmarks_list,thumb_index_dist):
            im1= pyautogui.screenshot()
            label =random.randint(1,1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame,'Screenshot Taken',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
          # Sleep Windows
        elif is_sleep(landmarks_list,thumb_index_dist):
            cv2.putText(frame,'Sleep Windows',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
            os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0")
            
          # ShutDown Windows
        elif is_shutdown(landmarks_list,thumb_index_dist):
             os.system("shutdown /s /t 0")
             # Display a message on the screen before shutdown
             cv2.putText(frame, 'Shutting Down...', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

      
    

       
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    
    try:
        while cap.isOpened():  # Corrected method name
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)
            landmarks_list =[]
            if processed.multi_hand_landmarks:
                hand_landmarks= processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks,mpHands.HAND_CONNECTIONS)
                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x,lm.y))
                    
            detect_gestures(frame,landmarks_list,processed) 
                    
                
            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        try:
            cv2.destroyAllWindows()  # Safely handle the potential error
        except cv2.error as e:
            print(f"Error closing windows: {e}")

if __name__ == '__main__':
    main()



C:\Users\Guess\AppData\Local\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
